In [71]:
import torch
import torchvision
from torchvision import transforms
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
device = ['cuda'] if torch.cuda.is_available() else ['cpu']

# load data

### 因為xml檔案讀取後是 object ，所以需要用 beautifulSoup 套件轉成 text
### 需要轉換的項目有 boxes , label
### 最後將其輸出成已經是 tensor 的 dic

In [72]:
def get_box(obj):
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)
    return [xmin,ymin,xmax,ymax]

def get_label(obj):
    if obj.find('name').text =='with_mask' :
        return 1 
    elif obj.find('name').text =='without_mask':
        return 2 
    return 0

In [73]:
def obj_to_target(image_id , file):
    with open (file) as f:
        data = f.read()
        soup = BeautifulSoup(data,'xml')
        objects = soup.find_all('object')
        obj_len = len(objects)
        boxes = []
        labels = []
        for i in objects:
            boxes.append(get_box(i))
            labels.append(get_label(i))

        boxes = torch.as_tensor(boxes,dtype = torch.float32)
        labels = torch.as_tensor(labels,dtype = torch.float32)
        image_id = torch.as_tensor([image_id],dtype = torch.float32)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        return target

In [74]:
imgs = list(sorted(os.listdir("images/")))
labels = list(sorted(os.listdir("annotations/")))

### Create DataSet

In [75]:
class MaskDataset(object):
    def __init__(self,trasforms):
        self.trasforms = trasforms
        self.imgs = list(sorted(os.listdir("images/")))
    
    def __getitem__(self,idx):
        file_image = "maksssksksss" + idx + ".png"
        file_labels = "maksssksksss" + idx + ".xml"
        image_path = os.path.join("images",file_image)
        label_path = os.path.join("annotations",file_labels)
        img = Image.open(image_path).covert('RGB')
        target = obj_to_target(idx, label_path)

        if self.trasforms != None:
            img = self.trasforms(img)
        
        return img , target
    def __len__(self):
        return len(self.imgs)

In [80]:
data_transform = transforms.Compose([
        transforms.ToTensor(), 
    ])

def collate_size(batch):
    return tuple(zip(batch))

In [79]:
dataset = MaskDataset(data_transform)
data_loader = torch.utils.data.DataLoader(dataset,batch_size=4,collate_fn=collate_size)

# model